In [ ]:
import sys
sys.path.insert(0, '/work/greenhouse-simulator-2/')

from datetime import datetime, timezone
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import simps
from numpy import trapz

from helpers.types import *
from helpers.data_prep import *
from helpers.solar_conversions import *
from greenhouse.structure.structure import *

## Generate curve real-time

In [ ]:
def get_curve_configs(
    sun_ppfd: umol_per_m2=None,
    led_ppfd: umol_per_m2=None,
    rotation_period: s=600,
    angle_exposed_to_sun: deg=180,
    angle_exposed_to_single_led: deg=90,
    inter_led_angle: deg=45,
    resolution=1000,
):
    """
    Creates the superposition of exposure curves of each individual light source to get
    the total irradiance of a plant.

    Parameters
    ----------
    sun_ppfd : umol_per_m2 | umol_per_m2[]
        Irradiance of sunlight (on a vertical surface perpendicular to the suns rays).
        If a list, length should equal to rotation_period * resolution.
    led_ppfd : umol_per_m2 | umol_per_m2[]
        Irradiance from a single light fixture. 
        If a list, length should equal to rotation_period * resolution.
    rotation_period : s
        Time it takes for the barrel to do a full rotation.
    angle_exposed_to_sun : deg
        Central angle between 2 radii that points to the start and end points of the barrel surface that
        is exposed to the sun.
    angle_exposed_to_single_led : deg
        Central angle between 2 radii that points to the start and end points of the barrel surface that
        is exposed to light from a single fixture.
    inter_led_angle : deg
        Angle between 2 radii that points to the center points of 2 adjacent light fixtures.
    resolution : int
        Number of points within one peak.
    """

    sun_azimuth: deg = 0

    sun_start: deg = sun_azimuth - angle_exposed_to_sun / 2
    led_1_start: deg = 180 - inter_led_angle - angle_exposed_to_single_led / 2
    led_2_start: deg = led_1_start + inter_led_angle
    led_3_start: deg = led_2_start + inter_led_angle

    curves = {
        "sun": [sun_start, angle_exposed_to_sun],
        "led_1": [led_1_start, angle_exposed_to_single_led],
        "led_2": [led_2_start, angle_exposed_to_single_led],
        "led_3": [led_3_start, angle_exposed_to_single_led]
    }

    if sun_start < 0:
        curves["sun"] = [sun_start, angle_exposed_to_sun]
        curves["sun_next"] = [360 - abs(sun_start), angle_exposed_to_sun]

    return curves

In [ ]:
def get_ppfd_of_light_source(x, xmax, ymax):
    return np.sin(math.pi * x / xmax) * ymax

In [ ]:
def get_total_ppfd(t, angle, sun_ppfd, led_ppfd, light_sources, rotation_period=600):
    """
    Parameters
    ----------
    t : int
        Seconds since the start of the last rotation
    """

    def angle_to_sec(angle: deg) -> s:
        return angle / 360 * rotation_period

    ppfd_of_current_t = 0
    for light_source_name in light_sources:
        x_start, length = light_sources[light_source_name]

        # Convert angles to seconds
        x_start_s = angle_to_sec(x_start)
        length_s = angle_to_sec(length)

        intensity = led_ppfd if "led" in light_source_name else sun_ppfd

        # Check which light sources can reach the current plant
        if x_start <= angle and angle <= x_start + length:
            y = get_ppfd_of_light_source(t - x_start_s, length_s, intensity)
            ppfd_of_current_t += y
    return ppfd_of_current_t

In [ ]:
def get_sun_and_led_ppfd(t, angle, sun_ppfd, led_ppfd, light_sources, rotation_period=600):
    """
    Parameters
    ----------
    t : int
        Seconds since the start of the last rotation
    """

    def angle_to_sec(angle: deg) -> s:
        return angle / 360 * rotation_period

    ppfd_of_current_t_sun = 0
    ppfd_of_current_t_led = 0
    for light_source_name in light_sources:
        x_start, length = light_sources[light_source_name]

        # Convert angles to seconds
        x_start_s = angle_to_sec(x_start)
        length_s = angle_to_sec(length)

        intensity = led_ppfd if "led" in light_source_name else sun_ppfd

        # Check which light sources can reach the current plant
        if x_start <= angle and angle <= x_start + length:
            y = get_ppfd_of_light_source(t - x_start_s, length_s, intensity)
            if "led" in light_source_name:
                ppfd_of_current_t_led += y
            else:
                ppfd_of_current_t_sun += y
    return ppfd_of_current_t_sun, ppfd_of_current_t_led

In [ ]:
def get_led_ppfd(seconds_since_midnight) -> umol_per_m2_s:
    if seconds_since_midnight < 4 * 60 * 60: # 6 AM
        return 0
    if 22 * 60 * 60 < seconds_since_midnight: # 10 PM
        return 0
    return 343

In [ ]:
rotation_period = 600
resolution = 1

light_sources = get_curve_configs(
    rotation_period=rotation_period, 
    resolution=resolution
)

results = []
for t in range(rotation_period):
    sun_ppfd: umol_per_m2_s = 1000
    led_ppfd: umol_per_m2_s = 343
    angle: deg = t / rotation_period * 360
    results.append(get_total_ppfd(t, angle, sun_ppfd, led_ppfd, light_sources, rotation_period=rotation_period))

plt.figure(figsize=(16,9))
plt.plot(results)

In [ ]:
coordinates = { "latitude": 38.7436883, "longitude": -9.1952227 }

In [ ]:
resample_period = "5s"
df = get_weather_data(date_from="2020-01-10", date_to="2020-01-11", resample_period=resample_period)

results_sun = []
results_led = []
for timestamp, row in df.iterrows():
    sun_azimuth = get_azimuth_angle(coordinates, timestamp)
    intensity_coeff = get_intensity_coeff(coordinates, timestamp, panel_tilt=90, panel_azimuth=sun_azimuth)

    seconds_since_midnight = int((timestamp - timestamp.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds())
    t = seconds_since_midnight % rotation_period
    angle: deg = t / rotation_period * 360
    sun_ppfd: umol_per_m2_s = irradiance_to_PPFD(intensity_coeff * row.solarradiation * 0.88)

    elevation_angle = get_elevation_angle(coordinates, timestamp)
    # print(timestamp.hour, intensity_coeff, row.solarradiation, sun_ppfd, elevation_angle)
    led_ppfd: umol_per_m2_s = get_led_ppfd(seconds_since_midnight)
    
    # results.append(get_total_ppfd(t, angle, sun_ppfd, led_ppfd, light_sources, rotation_period=rotation_period))
    sun_ppfd, led_ppfd = get_sun_and_led_ppfd(t, angle, sun_ppfd, led_ppfd, light_sources, rotation_period=rotation_period)
    results_sun.append(sun_ppfd)
    results_led.append(led_ppfd)


plt.figure(figsize=(16,9))
plt.plot(results_sun, label="sun")
plt.plot(results_led, label="led")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(16,9))
plt.plot(results_sun[7500:9000], label="sun")
plt.plot(results_led[7500:9000], label="led")
plt.legend()
plt.show()

In [ ]:
area_sun: umol_per_m2 = trapz(results_sun, dx=pd.Timedelta(resample_period).seconds)
area_led: umol_per_m2 = trapz(results_led, dx=pd.Timedelta(resample_period).seconds)
dli_sun: mol_per_m2_day = area_sun / 1e6
dli_led: mol_per_m2_day = area_led / 1e6
print("Sun:", round(dli_sun, 2))
print("LED:", round(dli_led, 2))
print("Total:", round(dli_sun + dli_led, 2))


In [ ]:
timestamp = datetime(2020, 6, 21, 13, 0, 0, tzinfo=timezone.utc)
sun_azimuth = get_azimuth_angle(coordinates, timestamp)
intensity_coeff = get_intensity_coeff(coordinates, timestamp, panel_tilt=90, panel_azimuth=sun_azimuth)
print(sun_azimuth)
print(intensity_coeff)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>